# **Multiclass Classification**

**Goal**: Using images of crop diseases from Uganda to build a convolutional neural network and train it to classify images into one of the five classes.

**Objectives**
- Convert images from grayscale to RGB
- Resize images 
- Normalize data
- Create a transformation pipeline to standardize images for training
- create a Convolutional Neural Network
- Train the network to do multiclass classification
- Identify overfitting. 